### 1) Importing Libraries

#### Standard Libraries

In [1]:
# ! source '/home/xicocaio/venv/bin/activate'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from sklearn.datasets import load_files
import pickle
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelfelipemagnossaodepaula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Model Libraries

In [2]:
# Import Import RandomForestClassifier
from sklearn.svm import SVR

#### Downloading Files from Google Drive

In [3]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

import os

In [4]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [5]:
# Downloading Headlines train and Test Data from google drive
# PS: the traing data is public and was downloaded to drive from:
#        https://bitbucket.org/ssix-project/semeval-2017-task-5-subtask-2/raw/46143dc3c0c28b9286b6d1d424d1359c8f49d6cc/Headline_Trainingdata.json
#     while the test data is avaliable n the same repo, but without labels
#     the one used here was requested by e-mail, and contains labels

# files_list = {'full_dict.txt': '1_ZBh3oFm3CYDqG3IcVY-ukG0-vsiulTp',
#               'train.csv': '1Tjy8fxm63Yg9n7EemvJ_G_Fbs6Y6SQEC',
#               'test.csv': '1NAQTM-0ynAQJ8CjHZ1AaxuMIuvuYjrgq',
#               'SenVal_Functions.py': '1QgNS3191Q87tdXgyyJLrOqAlQf3wjki4'}

# for filename, gdrive_id in files_list.items():
#   exists = os.path.isfile('/content/' + filename)
  
#   if not exists:
#     drive.CreateFile({'id': gdrive_id}).GetContentFile(filename)
#     print('File {} was downloaded.'.format(filename))
#   else:
#     print('File {} not downloaded because already on system.'.format(filename))

#### SenVal Evaluation Metric

In [6]:
from SenVal_Functions import SenVal_metric
from sklearn.metrics import make_scorer

#New scoring function
scoring_SenVal = make_scorer(SenVal_metric)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/angelfelipemagnossaodepaula/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


####  NLP

In [7]:
from SenVal_Functions import NLP_pre_processing

#### Dictionaries

In [8]:
#Dictionary with all the words in the corpus and only these
with open("full_dict.txt", "rb") as fp:   # Unpickling
    full_dict = pickle.load(fp)

In [9]:
# Download dictionary WORDS from nltk
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /Users/angelfelipemagnossaodepaula/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### 2) Load files

In [10]:
#test
df_test = pd.read_csv('test.csv', index_col='id')
#train
df_train = pd.read_csv('train.csv', index_col='id')

### 3) Split dataset in " Title " and " Label "

In [11]:
#Train_Title
train_title = []
for n in df_train['title']:
    train_title.append(n)
    
#Train_Label
train_label = df_train['sentiment_range'].values 

In [12]:
#Test_Title
test_title = []
for n in df_test['title']:
    test_title.append(n)
      
#Test_Label
test_label = df_test['sentiment_range'].values

### 4) Text Preprocessing

In [13]:
# Applying the NLP_pre_processing function 

#Train
documents_train = NLP_pre_processing(train_title)
#Test
documents_test = NLP_pre_processing(test_title)

### 5) Converting Text to Numbers 

#### Dictionary Preparation

In [14]:
# Import dictionary WORDS from nltk
from nltk.corpus import words

In [15]:
# Convert dictionary to set and add it into" vocab "
vocab = set(words.words())

#### Bag of Words

In [16]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
#Train
vectorizer_train = CountVectorizer(max_features=1500, vocabulary= full_dict, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
train_title = vectorizer_train.fit_transform(documents_train).toarray()

In [18]:
#Test
vectorizer_test = CountVectorizer(max_features=1500, vocabulary= full_dict, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
test_title = vectorizer_test.fit_transform(documents_test).toarray()

#### Finding TFIDF

In [19]:
# Import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
#Train
tfidfconverter_train = TfidfTransformer()
train_title = tfidfconverter_train.fit_transform(train_title).toarray()

In [21]:
#Test
tfidfconverter_test = TfidfTransformer()
test_title = tfidfconverter_test.fit_transform(test_title).toarray()

### 6) Training and Testing Sets

In [22]:
# Import train_test_split
from sklearn.model_selection import train_test_split  

In [23]:
#Train and Validation
X_train, X_validation, y_train, y_validation = train_test_split(train_title, train_label, test_size=0.20, random_state=0) 

In [24]:
#Test
#X_test, X, y_test, y = train_test_split(test_title, test_label, test_size=0, shuffle=False)
X_test = test_title
y_test = test_label

In [65]:
X_train.shape

(913, 4235)

In [67]:
y_train.shape

(913,)

### 7) Grid Search for Parameter Selection

#### SVM algorithm 

In [25]:
# Set the classifier_1
classifier_1 = SVR() 

#### Specify Grid Search

In [26]:
# Import the Grid Search algorithm 
from sklearn.model_selection import GridSearchCV

In [59]:
#Grid Search parameters
# C_range = np.logspace(-2, 5, 13)
# gamma_range = np.logspace(-9, 3, 13)
# degree_range = np.arange(2,8,1)

C_range = np.logspace(-2, 5, 13)
gamma_range = np.logspace(-9, 1, 2)
degree_range = np.arange(2,3,1)

In [60]:
# Create a dictionary of parameters and their corresponding values for Random Forest algorithm
grid_param = {
   'kernel':['poly'],
   'degree':degree_range,
   'gamma':gamma_range ,
   'C':C_range
}

# #Create a dictionary of parameters and their corresponding values for Random Forest algorithm
# grid_param = {
#     'kernel': ['linear', 'rbf', 'sigmoid'],
#     'C': C_range,
#     'gamma': gamma_range,
#     'degree': degree_range
# }

In [61]:
#Create an instance of the GridSearchCV class 

# Apply in the real train !!!
# gd_sr = GridSearchCV(estimator=classifier_1,  
#                     param_grid=grid_param,
#                     scoring= neg_mean_squared_error',
#                     cv=10,
#                     n_jobs=-1)

gd_sr = GridSearchCV(estimator=classifier_1,  
                     param_grid=grid_param,
                     scoring='neg_mean_squared_error',
                     cv=2,
                     n_jobs=-1,
                     verbose=1)

In [62]:
#Fit method of the class and pass it the training and test set
gd_sr.fit(X_train, y_train)

Fitting 2 folds for each of 26 candidates, totalling 52 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done  52 out of  52 | elapsed:   40.1s finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'kernel': ['poly'], 'degree': array([2]), 'gamma': array([1.e-09, 1.e+01]), 'C': array([1.00000e-02, 3.83119e-02, 1.46780e-01, 5.62341e-01, 2.15443e+00,
       8.25404e+00, 3.16228e+01, 1.21153e+02, 4.64159e+02, 1.77828e+03,
       6.81292e+03, 2.61016e+04, 1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

#### Check the parameters that return the lowest Mean squared errer MSE

In [57]:
best_parameters = gd_sr.best_params_
print(best_parameters)

{'C': 0.01, 'degree': 2, 'gamma': 10.0, 'kernel': 'poly'}


#### Find the Mean squared errer (MSE) obtained using the best parameters.

In [58]:
best_result = -gd_sr.best_score_
print(best_result)

0.1267464155758865


#### Specify Grid Search 2

In [0]:
#Grid Search parameters
C_range = np.logspace(-2, 5, 13)
gamma_range = np.logspace(-9, 1, 13)
degree_range = np.arange(2,8,1)


In [0]:
#Create a dictionary of parameters and their corresponding values for Random Forest algorithm
grid_param = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': C_range,
    'gamma': gamma_range,
    'degree': degree_range
}

### 9) Predicting Sentiment

#### Bag of Words

In [0]:
#Train
vectorizer_train = CountVectorizer(max_features=1500, vocabulary= vocab, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
train_title = vectorizer_train.fit_transform(documents_train).toarray()

In [0]:
#Test
vectorizer_test = CountVectorizer(max_features=1500, vocabulary= vocab, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
test_title = vectorizer_test.fit_transform(documents_test).toarray()

#### Finding TFIDF

In [0]:
#Train
tfidfconverter_train = TfidfTransformer()
train_title = tfidfconverter_train.fit_transform(train_title).toarray()

In [0]:
#Test
tfidfconverter_test = TfidfTransformer()
test_title = tfidfconverter_test.fit_transform(test_title).toarray()

#### Training Using the full " train dataset " 

In [0]:
#Test
X_train_full = train_title
y_train_full = train_label

In [55]:
# Set the classifier_2
classifier_2 = SVR(kernel='poly', degree= 3, gamma= 10, C= 1)
classifier_2.fit(X_train_full, y_train_full)


SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=10,
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

#### Prediction

In [57]:
y_pred = classifier_2.predict(X_test)

ValueError: ignored

### 10) Evaluating the Model

#### Ordinary metrics

In [0]:
# Import Import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('Mean Absolute Error:', mean_absolute_error(y_test,y_pred))  
print('Mean Squred Error:', mean_squared_error(y_test,y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))  

#### SanVal metric

In [0]:
#Reshape matrix
#Predict
y_pred=y_pred.reshape(1,-1)
#Label
y_test=y_test.reshape(1,-1)

In [0]:
SenVal_metric(y_test, y_pred)

### Graphics


### Validatio Curve

#### GAMA

In [0]:
#Import Validation Curve
from SenVal_Functions import plot_validation_curve

In [0]:
#Set parameters range
#estimator = SVR()
estimator = classifier_2
title = "Validation Curve with SVR"
x_label = r"$\gamma$"
param_name='gamma'
ylim = (0,1)
gamma_range = np.logspace(-9, 3, 13)
cv=2
n_jobs=-1

In [0]:
#Apply Validation Curve
plot_validation_curve(estimator, title, x_label, X_train_full, y_train_full, param_name, gamma_range, ylim, cv, n_jobs)

#### C

In [0]:
#Set parameters range
#estimator = SVR()
estimator = classifier_2
title = "Validation Curve with SVR"
x_label = r"C"
param_name='C'
ylim = (0,1)
C_range = np.logspace(-2, 5, 13)
cv=2
n_jobs=-1

In [0]:
#Apply Validation Curve
plot_validation_curve(estimator, title, x_label, X_train_full, y_train_full, param_name, C_range, ylim, cv, n_jobs)

#### Degree

In [0]:
#Set parameters range
#estimator = SVR()
estimator = classifier_2
title = "Validation Curve with SVR"
x_label = r"degree"
param_name='degree'
ylim = (0.1,0.5)
degree_range = np.arange(2,8,1)
cv=2
n_jobs=-1

In [0]:
#Apply Validation Curve
plot_validation_curve(estimator, title, x_label, X_train_full, y_train_full, param_name, degree_range, ylim, cv, n_jobs)

### Learning Curve

In [0]:
# Import curve/plot
from SenVal_Functions import plot_learning_curve

In [0]:
# Set the of learn curve plot
estimator = SVR(kernel='linear')
title = r"Learning Curves (SVR, Linear kernel, $\gamma=0.001$)"
ylim=(0,0.2)
cv=2
n_jobs=-1


In [0]:
#Apply learning Curve
plot_learning_curve(estimator, title, X_train_full, y_train_full, ylim, cv, n_jobs)

### 13) Saving and Loading the Model

In [0]:
with open('SVM_Regrassion_2', 'wb') as picklefile:
    pickle.dump(classifier_2,picklefile)

In [0]:
    #Load Model
#with open('text_classifier', 'rb') as training_model:  
    #model = pickle.load(training_model)